# Predict poverty from space

Folder organization :

* LICENSE  
* README.md
* Gaussian_Process.ipynb
* 0_data.ipynb
* data/ 
    *    daytime_images/  
    *    nightlights_intensities/  
        * F182010.v4d_web.stable_lights.avg_vis.tif  
    *    surveys/
        * DHS/
            * Rwanda_2010/
                * RWGE61FL.dbf
                * RWHR61FL.DTA
            * ...
        * LSMS-ISA/
            * ...
* models/
* papers/

In [1]:
import pandas as pd
import numpy as np
from simpledbf import Dbf5

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [2]:
wealth_file = "./data/surveys/DHS/Rwanda_2010/RWHR61FL.DTA"
geo_file = "./data/surveys/DHS/Rwanda_2010/RWGE61FL.dbf"

## 1) Import wealth data

Le premier fichier (RWHR61FL.DTA) contient les résultats des sondages. Deux colonnes nous intéressent plus particulièrement : HV001 est un identifiant qui fait le lien avec les données géographiques et HV271 contient un indice de richesse de la région.

In [3]:
df_wealth = pd.read_stata(wealth_file)

In [4]:
result = df_wealth.groupby('hv001')['hv271'].median().reset_index()
result['hv271'] /= 100000.
print(result.columns)

Index(['hv001', 'hv271'], dtype='object')


## 2) Import geographical data

Le second fichier contient les coordonnées géographiques des sondages. Nous nous intéressons notamment à la colonne DHSCLUST qui fait le lien avec les clusters du fichier précédent ainsi qu'aux colonnes LATNUM et LONGNUM qui nous permettront d'obtenir l'intensité lumineuse de ces zones géographiques ainsi que les images satellites de jour.

In [5]:
dbf = Dbf5(geo_file)

In [6]:
df_geo = dbf.to_dataframe()

In [8]:
result2 = df_geo[['DHSCLUST', 'LATNUM', 'LONGNUM']]
result2['DHSCLUST'] = result2['DHSCLUST'].astype(int)
print(result2.columns)

Index(['DHSCLUST', 'LATNUM', 'LONGNUM'], dtype='object')


/home/ridouane/anaconda3/envs/Predict-poverty/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## 3) Merge wealth and geographical data

L'objectif de cette section est d'obtenir un tableau de données dans lequel on a l'indice de richesse en fonction des coordonnées géographiques.

In [9]:
final_result = pd.merge(result, result2, how='inner', left_on='hv001', right_on='DHSCLUST')[['DHSCLUST', 'LATNUM', 'LONGNUM', 'hv271']]
print(final_result.columns)

Index(['DHSCLUST', 'LATNUM', 'LONGNUM', 'hv271'], dtype='object')


In [10]:
final_result = final_result.rename(columns={'DHSCLUST': 'cluster', 'LATNUM': 'latitude', 'LONGNUM': 'longitude', 'hv271': 'wealth_index'})

In [83]:
print(final_result.columns)

Index(['cluster', 'latitude', 'longitude', 'wealth_index'], dtype='object')


## 4) Import nightlights intensities data

In [12]:
night_file = './data/nightlights_intensities/F182010.v4d_web.stable_lights.avg_vis.tif'

In [13]:
from osgeo import gdal, ogr, osr

In [14]:
dataset = gdal.Open(night_file, gdal.GA_ReadOnly)
print(type(dataset))

<class 'osgeo.gdal.Dataset'>


In [21]:
print("Driver: {}/{}".format(dataset.GetDriver().ShortName,
                             dataset.GetDriver().LongName))
print("Size is {} x {} x {}".format(dataset.RasterXSize,
                                    dataset.RasterYSize,
                                    dataset.RasterCount))
print("Projection is {}".format(dataset.GetProjection()))
(upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = dataset.GetGeoTransform()
print(upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size)
if geotransform:
    print("Origin = ({}, {})".format(upper_left_x, upper_left_y))
    print("Pixel Size = ({}, {})".format(x_size, y_size))

Driver: GTiff/GeoTIFF
Size is 43201 x 16801 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]
-180.00416666665 0.0083333333 0.0 75.00416666665 0.0 -0.0083333333
Origin = (-180.00416666665, 75.00416666665)
Pixel Size = (0.0083333333, -0.0083333333)


In [16]:
band = dataset.GetRasterBand(1)
print("Band Type={}".format(gdal.GetDataTypeName(band.DataType)))
      
min = band.GetMinimum()
max = band.GetMaximum()
if not min or not max:
    (min,max) = band.ComputeRasterMinMax(True)
print("Min={:.3f}, Max={:.3f}".format(min,max))
      
if band.GetOverviewCount() > 0:
    print("Band has {} overviews".format(band.GetOverviewCount()))
      
if band.GetRasterColorTable():
    print("Band has a color table with {} entries".format(band.GetRasterColorTable().GetCount()))

Band Type=Byte
Min=0.000, Max=63.000


In [18]:
band = dataset.GetRasterBand(1)
band_array = band.ReadAsArray()
print(band_array.mean())
rows, cols = np.shape(band_array)
print(rows, cols)

0.6435769258995662
16801 43201


In [28]:
x_size = 1.0 / int(round(1 / float(x_size)))
y_size = - x_size
y_index = np.arange(band_array.shape[0])
x_index = np.arange(band_array.shape[1])
top_left_x_coords = upper_left_x + x_index * x_size
top_left_y_coords = upper_left_y + y_index * y_size
centroid_x_coords = top_left_x_coords + (x_size / 2)
centroid_y_coords = top_left_y_coords + (y_size / 2)

In [97]:
def get_cell_idx(lon, lat, top_left_x_coords, top_left_y_coords):
    lon_idx = np.where(top_left_x_coords < lon)[0][-1]
    lat_idx = np.where(top_left_y_coords > lat)[0][-1]
    return lon_idx, lat_idx

def get_nightlight_feature(sample):
    ind, wealth_ind, x, y = sample
    lon_idx, lat_idx = get_cell_idx(x, y, top_left_x_coords, top_left_y_coords)
    # Select the 10 * 10 pixels
    left_idx = lon_idx - 5
    right_idx = lon_idx + 4
    up_idx = lat_idx - 5
    low_idx = lat_idx + 4
    luminosity_100 = []
    for i in range(left_idx, right_idx + 1):
        for j in range(up_idx, low_idx + 1):
            # Get the luminosity of this pixel
            luminosity = band_array[j, i]
            luminosity_100.append(luminosity)
    luminosity_100 = np.asarray(luminosity_100)
    max_ = np.max(luminosity_100)
    min_ = np.min(luminosity_100)
    mean_ = np.mean(luminosity_100)
    median_ = np.median(luminosity_100)
    std_ = np.std(luminosity_100)
    return ind, wealth_ind, x, y, max_, min_, mean_, median_, std_

In [81]:
sample = 29.684726, -2.532818
print(get_nightlight_feature((sample)))

25162 9304
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6]
(6, 0, 0.06, 0.0, 0.5969924622639721)


In [82]:
x = int((-2.532818+180-75)*16800/180) - 5
y = int((29.684726+360-180)*43200/360) - 5
print(x, y)
#x = 9304
#y = 25162
res = band_array[9304, 25162]
for i in range(x-5, x+5):
    for j in range(y-5, y+5):
        res = band_array[i, j]

9558 25157


In [103]:
for lat, long in final_result[['latitude', 'longitude']].values:
    sample = 1, 2, lat, long
    ind, wealth, lt, lg, max_, min_, mean_, median_, std_ = get_nightlight_feature((sample))

In [108]:
dataset_night = final_result.apply(lambda x: get_nightlight_feature([x['cluster'], x['wealth_index'], 
                                                                     x['longitude'], x['latitude']]), axis=1)

In [109]:
def example(x):
    x['p1'] = x['num']**2
    x['p2'] = x['num']**3
    x['p3'] = x['num']**4
    return x

def get_nightlight_feature(df):
    ind, wealth_ind, x, y = sample
    lon_idx, lat_idx = get_cell_idx(x, y, top_left_x_coords, top_left_y_coords)
    
    # Select the 10 * 10 pixels
    left_idx = lon_idx - 5
    right_idx = lon_idx + 4
    up_idx = lat_idx - 5
    low_idx = lat_idx + 4
    
    luminosity_100 = []
    for i in range(left_idx, right_idx + 1):
        for j in range(up_idx, low_idx + 1):
            # Get the luminosity of this pixel
            luminosity = band_array[j, i]
            luminosity_100.append(luminosity)
    luminosity_100 = np.asarray(luminosity_100)
    
    max_ = np.max(luminosity_100)
    min_ = np.min(luminosity_100)
    mean_ = np.mean(luminosity_100)
    median_ = np.median(luminosity_100)
    std_ = np.std(luminosity_100)
    
    x['max'] = max_
    x['min'] = min_
    x['mean'] = mean_
    x['median'] = median_
    x['std'] = std_
    return x

dataset_night = final_result.apply(get_nightlight_feature, axis=1)

0      (1.0, -0.531405, 29.684726, -2.532818, 6, 0, 0...
1      (2.0, -0.40983, 30.310689, -1.833858, 0, 0, 0....
2      (3.0, -0.478115, 29.478298, -1.888155, 0, 0, 0...
3      (4.0, -0.43596, 30.521692, -2.366763, 0, 0, 0....
4      (5.0, -0.44948, 30.018541, -2.171266, 0, 0, 0....
5      (6.0, -0.11265, 30.114591, -2.036917, 26, 0, 6...
6      (7.0, -0.39962, 29.799135, -1.768636, 0, 0, 0....
7      (8.0, -0.19558, 29.967649, -1.769425, 0, 0, 0....
8      (9.0, 2.39554, 30.111135, -1.974834, 62, 7, 36...
9      (10.0, 0.05646, 30.310015, -2.038221, 0, 0, 0....
10     (11.0, -0.233975, 30.651838, -2.021018, 0, 0, ...
11     (12.0, -0.45845, 29.921206, -2.212707, 0, 0, 0...
12     (13.0, -0.643445, 29.413435, -2.22724, 0, 0, 0...
13     (14.0, -0.50153, 29.512758, -1.912341, 0, 0, 0...
14     (15.0, -0.49069, 29.631617, -1.997883, 0, 0, 0...
15     (16.0, -0.54898, 30.824889, -2.072847, 0, 0, 0...
16     (17.0, -0.41252, 29.813837, -2.116178, 7, 0, 0...
17     (18.0, -0.41903, 29.5558

## 5) Merge wealth, geographical and nightlights intensities data

L'objectif de cette partie est d'obtenir un tableau de données dans lequel on a, pour chaque région données par ses coordonnées, l'indice de richesse et l'indice de luminosité.

## 6) Try to predict wealth thanks to nightlights intensities

## 7) Download daytime images

## 8) Merge wealth, geographical, nightlights intensities and daytime images data

## 9) Try to predict wealth thanks to deep features from daytime images

## 10) Retrain VGG by predicting nightlights intensities thanks to daytime images

## 11) Try to predict wealth thanks to new deep features from daytime images